In [147]:
import pandas as pd
import numpy as np
import plotly.express as px
import statistics

In [285]:
file_list = ['1000m_2_Full_Data_data_2018.csv', '1000m_2_Full_Data_data_2019.csv', '1000m_2_Full_Data_data_2020.csv']

li = []

for filename in file_list:
    frame = pd.read_csv(filename)
    frame.columns = ['place_overall', 'place_gender', 'name', 'gender', 'sub20', 'best_time', 'result_month',
       'result_year', 'result_time', 'result_gender', 'sub20_flag', 'distance']
    li.append(frame)

frame_2017 = pd.read_csv('1000m_2_Full_Data_data_2017.csv')
frame_2017.columns = ['place_overall', 'place_gender', 'name', 'gender', 'sub20', 'best_time', 'result_month', 'result_time']
frame_2017['result_year'] = 2017

li.append(frame_2017)

df = pd.concat(li, axis=0, ignore_index=True)


In [366]:
df['result_year'].unique()

array([2017, 2018, 2019, 2020], dtype=int64)

In [363]:
df.head()

,place_overall,place_gender,name,gender,sub20,best_time,result_month,result_year,result_time,result_gender,sub20_flag,distance,result_month_year
3179,56,18,Шелике Аяна,Ж,NaN,00:20:23,February,2017,0 days 00:21:02,NaN,NaN,NaN,201702
3226,82,28,Поддубная Екатерина,Ж,NaN,00:21:25,February,2017,0 days 00:21:25,NaN,NaN,NaN,201702
3424,191,124,Науменко Денис,М,NaN,00:24:06,February,2017,0 days 00:27:51,NaN,NaN,NaN,201702
3428,194,125,Тищенко Юрий,М,NaN,00:24:12,February,2017,0 days 00:25:11,NaN,NaN,NaN,201702
3433,199,129,Нагорянский Сергей,М,NaN,00:24:25,February,2017,0 days 00:24:25,NaN,NaN,NaN,201702


In [294]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3653 entries, 0 to 3652
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   place_overall  3653 non-null   int64  
 1   place_gender   3653 non-null   int64  
 2   name           3653 non-null   object 
 3   gender         3653 non-null   object 
 4   sub20          852 non-null    object 
 5   best_time      3653 non-null   object 
 6   result_month   3653 non-null   object 
 7   result_year    3653 non-null   int64  
 8   result_time    3653 non-null   object 
 9   result_gender  3060 non-null   object 
 10  sub20_flag     746 non-null    object 
 11  distance       3060 non-null   float64
dtypes: float64(1), int64(3), object(8)
memory usage: 342.6+ KB


In [295]:
df.result_time = pd.to_timedelta(df['result_time'])
df['result_month_year'] = df.result_year * 100 + pd.to_datetime ( df.result_month, format='%B').dt.month 

Longest finish ever:

In [297]:
df['result_time'].max()

Timedelta('0 days 01:02:00')

Best result ever:

In [361]:
df[ df['result_time'] == df['result_time'].min() ]

,place_overall,place_gender,name,gender,sub20,best_time,result_month,result_year,result_time,result_gender,sub20_flag,distance,result_month_year
2660,1,1,Кузяев Юрий,М,SUB20,00:13:50,September,2020,0 days 00:13:50,00:13:50,sub20,1000.0,202009


Mean finish time:

In [362]:
df['result_time'].mean()

Timedelta('0 days 00:24:19.602792225')

In [300]:
df = df.sort_values(by='result_month_year')

In [324]:
def result_improvements(df, distance):
    all_improvements = []
    for athlete in df['name'].unique():
        athlete_results = df[df['name'] == athlete]
        athlete_results.reset_index(drop=True, inplace=True)
        athlete_improvement = []
        for i in range(0,len(athlete_results)):
            if i < len(athlete_results)-distance-1:
                athlete_improvement.append( (athlete_results.loc[i+distance,'result_time'] - athlete_results.loc[i+distance+1,'result_time']).total_seconds() )
                all_improvements.append( (athlete_results.loc[i+distance,'result_time'] - athlete_results.loc[i+distance+1,'result_time']).total_seconds() )
#        if athlete_improvement != []:
#            print(1)
#        else:
#            print( np.mean(athlete_improvement) )
#            all_improvements.append(athlete_improvement)
#    print(len(all_improvements))
    return all_improvements

In [448]:
#result_improvements(df[df['name'] == 'Ковальчук Юлия'], 0)
#result_improvements(df[df['name'] == 'Бодрова Анастасия'], 0)
#result_improvements(df[df['name'] == 'Исаев Артур'],0)

improvements_over_months = []

for i in range(0,10):
    improvement=result_improvements(df[df['gender'] == 'М'], i)
    improvements_over_months.append([i+1, np.mean(improvement)])
    print('Improvement in males: ',i+1,' month is {:.2f}'.format(np.mean(improvement)),'seconds and number of datapoints: ', len(improvement) )

print('and what about females?\n')

improvements_over_months = []

for i in range(0,10):
    improvement=result_improvements(df[df['gender'] == 'Ж'], i)
    improvements_over_months.append([i+1, np.mean(improvement)])
    print(' ',i+1,' month is {:.2f}'.format(np.mean(improvement)))
    #,'seconds and number of datapoints: ', len(improvement) )
#    print('Improvement in females: ',i+1,' month is {:.2f}'.format(np.mean(improvement)),'seconds and number of datapoints: ', len(improvement) )


#improvement_first_month = result_improvements(df, 0)
#improvement_second_month = result_improvements(df, 1)
#improvement_third_month = result_improvements(df, 2)

#print('Improvements in a first month: ', improvement_first_month)
#print('Improvements in a second month: ', improvement_second_month)
#print('Improvements in a third month: ', improvement_third_month)


Improvement in males:  1  month is 49.02 seconds and number of datapoints:  921
Improvement in males:  2  month is 34.49 seconds and number of datapoints:  588
Improvement in males:  3  month is 25.18 seconds and number of datapoints:  381
Improvement in males:  4  month is 24.03 seconds and number of datapoints:  251
Improvement in males:  5  month is 12.53 seconds and number of datapoints:  168
Improvement in males:  6  month is 11.77 seconds and number of datapoints:  110
Improvement in males:  7  month is 9.88 seconds and number of datapoints:  67
Improvement in males:  8  month is 10.00 seconds and number of datapoints:  36
Improvement in males:  9  month is -3.53 seconds and number of datapoints:  19
Improvement in males:  10  month is 9.78 seconds and number of datapoints:  9
and what about females?

  1  month is 63.73
  2  month is 38.32
  3  month is 27.02
  4  month is 24.39
  5  month is 15.31
  6  month is 10.17
  7  month is 7.00
  8  month is 10.42
  9  month is 7.00
  1

In [258]:
df.head()

,place_overall,place_gender,name,gender,sub20,best_time,result_month,result_year,result_time,result_gender,sub20_flag,distance,result_month_year
0,166,54,Бодрова Анастасия,Ж,NaN,00:22:26,January,2019,0 days 00:23:57,00:23:57,NaN,1000,201901
722,52,38,Копылов Денис,М,SUB20,00:19:15,January,2019,0 days 00:24:15,00:24:15,sub20,1000,201901
721,46,35,Фаткуллин Камиль,М,SUB20,00:19:05,January,2019,0 days 00:19:05,00:19:05,sub20,1000,201901
720,33,24,Самохвалов Денис,М,SUB20,00:18:27,January,2019,0 days 00:18:27,00:18:27,sub20,1000,201901
719,11,8,Шевченко Александр,М,SUB20,00:16:42,January,2019,0 days 00:16:42,00:16:42,sub20,1000,201901


Athletes sorted by number of results:

In [381]:
df.groupby(['name', 'gender'])['name'].count().sort_values(ascending=False).head(20)

name                  gender
Бобров Дмитрий        М         15
Касьянова Маргарита   Ж         14
Вдовина Светлана      Ж         13
Ермошина Светлана     Ж         12
Шевченко Александр    М         12
Середкина Дина        Ж         12
Палкина Анна          Ж         12
Крамаренко Наталия    Ж         12
Трегубова Ольга       Ж         11
Дьяконова Татьяна     Ж         11
Тарнавская Наталья    Ж         11
Ивин Сергей           М         11
Бондаренко Галина     Ж         11
Чачко Агата           Ж         11
Константинов Дмитрий  М         11
Прядкина Наталья      Ж         10
Пчелинцева Анастасия  Ж         10
Ехтарян Виктория      Ж         10
Михайлова Галина      Ж         10
Родионов Евгений      М         10
Name: name, dtype: int64

Breakdown results for male:

In [454]:
df_male = df[df['gender'] == 'М']

results_count_by_time = df_male.groupby(pd.Grouper(freq='15s', key='result_time'))['place_overall'].count()
results_count_by_time.index = results_count_by_time.index.total_seconds() / 60
fig = px.line(results_count_by_time)
fig.show()

Breakdown results for female:

In [453]:
df_female = df[df['gender'] == 'Ж']


results_count_by_time = df_female.groupby(pd.Grouper(freq='30s', key='result_time'))['place_overall'].count()
results_count_by_time.index = results_count_by_time.index.total_seconds() / 60
#results_count_by_time
fig = px.line(results_count_by_time)
fig.show()

Let's see mean values for male / female over all results:

In [443]:
mean_male_sub20 = df_male[df_male['result_time'] < '01:20:00' ]['result_time'].mean().total_seconds() / 60
mean_female_sub20 = df_female[df_female['result_time'] < '01:20:00' ]['result_time'].mean().total_seconds() / 60

print('Mean result for male: {:.2f}'.format(mean_male_sub20))
print('Mean result for female: {:.2f}'.format(mean_female_sub20))
print('That is {:.2f} minutes difference'.format(mean_female_sub20 - mean_male_sub20))

Mean result for male: 22.84
Mean result for female: 25.65
That is 2.81 minutes difference


What happens if we look only for top athletes? Let's filter by sub20 results and compare mean time between male / female:

In [445]:
mean_male_sub20 = df_male[df_male['result_time'] < '00:20:00' ]['result_time'].mean().total_seconds() / 60
mean_female_sub20 = df_female[df_female['result_time'] < '00:20:00' ]['result_time'].mean().total_seconds() / 60

print('Mean sub20 result for male: {:.2f}'.format(mean_male_sub20))
print('Mean sub20 result for female: {:.2f}'.format(mean_female_sub20))
print('That is only {:.2f} minutes difference!'.format(mean_female_sub20 - mean_male_sub20))

Mean sub20 result for male: 18.18
Mean sub20 result for female: 18.52
That is only 0.34 minutes difference!


What about number of results per male / female?

In [426]:
#athlethes_by_resultcount = df.groupby(['name', 'gender'])['name'].count()

#male_count = 0
#female_count = 0
#male_results = 0
#female_results = 0

#athlethes_by_resultcount = athlethes_by_resultcount.to_frame()
#athlethes_by_resultcount.columns = ['results_count']
#athlethes_by_resultcount = athlethes_by_resultcount.reset_index()
#athlethes_by_resultcount[ athlethes_by_resultcount['gender'] =='']

#for i in range(0, len(athlethes_by_resultcount)):
#    print(athlethes_by_resultcount.index)
#    if athlethe['gender'] == 'М':
#        male_count +=1
#        male_results += athlete
#    else:
#        female_count +=1

attempts_per_male = df[df['gender'] == 'М']['name'].count() / df[df['gender'] == 'М'].groupby('name').count().count()['gender']
print ('Average attemps per unique male ', attempts_per_male)
attempts_per_female = df[df['gender'] == 'Ж']['name'].count() / df[df['gender'] == 'Ж'].groupby('name').count().count()['gender']
print ('Average attemps per unique female ', attempts_per_female)



Average attemps per unique male  2.149812734082397
Average attemps per unique female  2.337772397094431


In [437]:
df_female[df_female['result_time'] < '00:20:00' ]['name'].count()

177

In [438]:
df_male[df_male['result_time'] < '00:20:00' ]['name'].count()

465